# Advent of Code 2024

## Day 1

### Part One

In [1]:
with open("input/01", encoding="utf8") as input_data:
    list1, list2 = zip(*(line.split() for line in input_data))

result: int = sum(abs(int(a) - int(b)) for a, b in zip(sorted(list1), sorted(list2)))

print(result)

1341714


### Part Two

In [2]:
similarity = sum(int(a) * list2.count(a) for a in list1)

print(similarity)

27384707


## Day 2

### Part One

In [3]:
with open("input/02", encoding="utf-8") as input_data:
    reports: list[list[int]] = [[int(x) for x in line.split()] for line in input_data]

def check_safe_report(report: list[int]) -> bool:
    variations: list[int] = [b - a for (a, b) in zip(report, report[1:])]

    # Check if all variations are the same sign
    # (only need to check if min and max are same sign)
    same_direction: bool = min(variations) * max(variations) > 0

    # Check if no variation exceeds 3 
    no_big_variation: bool = all(abs(x) < 4 for x in variations)

    return same_direction and no_big_variation

safe_reports: int = sum(1 for report in reports if check_safe_report(report))

print(safe_reports)

526


### Part Two

In [4]:
def check_safe_report(report: list[int], retrying: bool = False) -> bool:

    direction: int = report[1] - report[0]

    for i, (a, b) in enumerate(zip(report, report[1:])):

        if not 1 <= a - b <= 3:
            return False if retrying else any((
                check_safe_report(report[:i] + report[i+1:], retrying=True),
                check_safe_report(report[:i+1] + report[i+2:], retrying=True)
            ))

    return True

safe_reports: int = sum(1 for report in reports if check_safe_report(report) or check_safe_report(report[::-1]))

print(safe_reports)

566


## Day 3

### Part One

In [5]:
import re

with open("input/03", encoding="utf-8") as input_data:
    program: str = input_data.read()

RE_MUL = re.compile(r"mul\((\d{1,3}),(\d{1,3})\)")

result: int = sum(int(a) * int(b) for (a, b) in RE_MUL.findall(program))

print(result)

191183308


### Part Two

In [6]:
RE_MUL_DO = re.compile(r"(do\(\))|(don't\(\))|(?:mul\((\d{1,3}),(\d{1,3})\))")

result: int = 0
enabled: bool = True

for do, dont, a, b in RE_MUL_DO.findall(program):
    if do:
        enabled = True
    elif dont:
        enabled = False
    elif a and b and enabled:
        result += int(a) * int(b)

print(result)

92082041


## Day 4

### Part One

In [7]:
from collections.abc import Iterable, Callable

with open("input/04", encoding="utf-8") as input_data:
    grid: list[list[str]] = [list(line) for line in input_data]

def join(s: Iterable[str]) -> str: return "".join(s)

directions: Callable[[int, int], tuple[int, int]] = [
    lambda x, y, i: [x, y + i],
    lambda x, y, i: [x, y - i],
    lambda x, y, i: [x - i, y],
    lambda x, y, i: [x - i, y + i],
    lambda x, y, i: [x - i, y - i],
    lambda x, y, i: [x + i, y],
    lambda x, y, i: [x + i, y + i],
    lambda x, y, i: [x + i, y - i],
]

total_occurences: int = 0

for i, line in enumerate(grid):
    for j, cell in enumerate(line):

        # Processing cell (i, j): for each direction
        # process 4 cells and check if the result makes 'XMAS'
        for advance in directions:
            result: str = join(
                grid[x][y]
                for k in range(4)
                if 0 <= (x := advance(i, j, k)[0]) < len(grid)
                and 0 <= (y := advance(i, j, k)[1]) < len(line)
            )
            if result == "XMAS":
                total_occurences += 1

print(total_occurences)

2496


### Part Two

In [8]:
total_occurences: int = 0

# Iterate over grid, don't check edge cells
for i, line in enumerate(grid[1:-1], start=1):
    for j, cell in enumerate(line[1:-1], start=1):

        if not cell == 'A': continue # Skip iteration

        # When a A is found, check if the two opposite corners makes "MAS" or "SAM"
        up_diag = join([grid[i + 1][j - 1], cell, grid[i - 1][j + 1]])
        down_diag = join([grid[i - 1][j - 1], cell, grid[i + 1][j + 1]])
        
        if up_diag in ("MAS", "SAM") and down_diag in ("MAS", "SAM"):
            total_occurences += 1

print(total_occurences)

1967


## Day 5

### Part One

In [9]:
from collections import defaultdict, namedtuple

# Page ordering rules: {page_number -> ([pages_before], [pages_after])}
# I suspect pages_after won't be needed, might delete later
page_rules = namedtuple("page_rules", ["pages_before", "pages_after"])
ordering_rules: dict[int, tuple[list[int], list[int]]] = defaultdict(lambda: page_rules(set(), set()))

with open("input/05", encoding="utf-8") as input_data:

    # Parsing page ordering rules
    while (line := input_data.readline()[:-1]):
        (page_before, page_after) = [int(x) for x in line.split("|")]
        ordering_rules[page_before].pages_after.add(page_after)
        ordering_rules[page_after].pages_before.add(page_before)

    # Parsing page updates
    updates: list[list[int]] = [[int(page) for page in line.split(",")] for line in input_data]

def check_update(update: list[int]) -> bool:
    """
    Checks if the update is printed correctly according to the page ordering rules.
    """
    return all(
        not any(later_page in ordering_rules[page].pages_before for later_page in update[i:])
        for (i, page) in enumerate(update)
    )

result = sum(update[len(update)//2] for update in updates if check_update(update))

print(result)

5064


### Part Two

In [10]:
from functools import cmp_to_key

incorrect_updates = [update for update in updates if not check_update(update)]

def sort_pages(page1: int, page2: int) -> int:
    """
    Comparator implementation for sorting the pages
    based on their ordering rules.
    """
    return -1 if page1 in ordering_rules[page2].pages_before else 1

result = sum(sorted(update, key=cmp_to_key(sort_pages))[len(update)//2] for update in incorrect_updates)

print(result)

5152
